In [ ]:
# --- Install necessary libraries ---
print("\n--- Installing libraries ---")
!pip install transformers torch pandas pillow -q

print("\nSetup is complete!")


--- Installing libraries ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.3 MB/s eta 0:00:00

Setup is complete!


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("banuprasadb/flickr30k")

print("Path to dataset files:", path)

100%|██████████| 4.08G/4.08G [00:58<00:00, 74.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/banuprasadb/flickr30k/versions/2


In [ ]:
import os

# --- Define the paths to your data ---
# This is based on the location you've identified.
DATA_PATH = "/root/.cache/kagglehub/datasets/banuprasadb/flickr30k/versions/2"
IMAGES_PATH = os.path.join(DATA_PATH, "Images") # Note the capital 'I'
CAPTIONS_PATH = os.path.join(DATA_PATH, "captions.txt")

# --- Verify that the paths are correct ---
print("--- Verifying file paths ---")
if os.path.exists(IMAGES_PATH) and os.path.exists(CAPTIONS_PATH):
    print("✅ Success: Images folder and captions file found.", CAPTIONS_PATH)
    # Print the first 5 items in the images folder to be sure
    print("Sample of files in Images folder:", os.listdir(IMAGES_PATH)[:5])
else:
    print("❌ Error: Could not find the Images folder or captions.txt at the specified path.")
    print("Let's see what is actually in your data directory:")
    !ls -l {DATA_PATH}

--- Verifying file paths ---
✅ Success: Images folder and captions file found. /root/.cache/kagglehub/datasets/banuprasadb/flickr30k/versions/2/captions.txt
Sample of files in Images folder: ['4351734575.jpg', '3155601377.jpg', '6086876085.jpg', '3532028205.jpg', '4966187934.jpg']


In [ ]:
import pandas as pd
import os

# --- Reuse the paths you successfully verified ---
DATA_PATH = "/root/.cache/kagglehub/datasets/banuprasadb/flickr30k/versions/2"
IMAGES_PATH = os.path.join(DATA_PATH, "Images") # Note the capital 'I'
CAPTIONS_PATH = os.path.join(DATA_PATH, "captions.txt")

# --- Load and process the captions file ---
print("--- Loading and preparing captions data ---")
try:
    # Read the captions.txt file. It's separated by a comma.
    # We will name the columns 'image' and 'caption'.
    df = pd.read_csv(CAPTIONS_PATH, sep=',', header=None, names=['image', 'caption'])

    # The first row might accidentally be a header like ('image', 'caption'), let's remove it if it is.
    if df.iloc[0]['image'] == 'image' and df.iloc[0]['caption'] == 'caption':
        df = df[1:].reset_index(drop=True)

    # Create the full path for each image file
    # This combines the IMAGES_PATH with the 'image' column from our dataframe
    df['image_path'] = df['image'].apply(lambda x: os.path.join(IMAGES_PATH, x))

    # --- Final Verification ---
    # Check if the first image path we created actually exists
    if os.path.exists(df['image_path'][0]):
        print("✅ Success: The data is loaded and image paths are correct.")
        print("\nHere is a sample of your prepared data:")
        print(df.head())
    else:
        print("❌ Error: The image paths could not be verified.")
        print(f"An example path that failed: {df['image_path'][0]}")

except FileNotFoundError:
    print(f"❌ CRITICAL ERROR: The captions file was not found at {CAPTIONS_PATH}")


--- Loading and preparing captions data ---
✅ Success: The data is loaded and image paths are correct.

Here is a sample of your prepared data:
            image                                            caption  \
0  1000092795.jpg   Two young guys with shaggy hair look at their...   
1  1000092795.jpg   Two young , White males are outside near many...   
2  1000092795.jpg   Two men in green shirts are standing in a yard .   
3  1000092795.jpg       A man in a blue shirt standing in a garden .   
4  1000092795.jpg            Two friends enjoy time spent together .   

                                          image_path  
0  /root/.cache/kagglehub/datasets/banuprasadb/fl...  
1  /root/.cache/kagglehub/datasets/banuprasadb/fl...  
2  /root/.cache/kagglehub/datasets/banuprasadb/fl...  
3  /root/.cache/kagglehub/datasets/banuprasadb/fl...  
4  /root/.cache/kagglehub/datasets/banuprasadb/fl...  


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

# --- Define the model we want to use ---
MODEL_NAME = "Salesforce/blip-image-captioning-base"

print("--- Loading the processor and the model ---")
# Load the processor
processor = BlipProcessor.from_pretrained(MODEL_NAME)

# Load the model
model = BlipForConditionalGeneration.from_pretrained(MODEL_NAME)

# --- Set the device ---
# We check if a GPU is available (which it should be in Colab) and set our model to use it.
# This will make the training process much faster.
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print(f"\n✅ Success: Processor and model have been loaded.")
print(f"The model has been moved to the '{device.upper()}' for processing.")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


--- Loading the processor and the model ---


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]


✅ Success: Processor and model have been loaded.
The model has been moved to the 'CPU' for processing.


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import os

# --- Re-define paths and load the DataFrame ---
# (This is just to make this cell runnable on its own after a session restart)
DATA_PATH = "/root/.cache/kagglehub/datasets/banuprasadb/flickr30k/versions/2"
IMAGES_PATH = os.path.join(DATA_PATH, "Images")
CAPTIONS_PATH = os.path.join(DATA_PATH, "captions.txt")

df = pd.read_csv(CAPTIONS_PATH, sep=',', header=None, names=['image', 'caption'])
if df.iloc[0]['image'] == 'image':
    df = df[1:].reset_index(drop=True)
df['image_path'] = df['image'].apply(lambda x: os.path.join(IMAGES_PATH, x))


# --- Create the Corrected Custom Dataset Class ---
class FlickrDataset(Dataset):
    """Custom PyTorch Dataset for our Flickr30k data."""
    def __init__(self, dataframe, processor):
        self.dataframe = dataframe
        self.processor = processor

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item = self.dataframe.iloc[idx]
        image = Image.open(item['image_path']).convert("RGB")
        caption = item['caption']

        encoding = self.processor(images=image, text=caption, padding="max_length", return_tensors="pt")

        # Squeeze the tensors to remove the batch dimension
        encoding = {key: val.squeeze() for key, val in encoding.items()}

        # --- THIS IS THE FIX ---
        # The model needs the input_ids to be passed as 'labels' for loss calculation
        encoding["labels"] = encoding["input_ids"]

        return encoding

print("✅ Corrected 'FlickrDataset' class has been defined.")

✅ Corrected 'FlickrDataset' class has been defined.


In [ ]:
from transformers import TrainingArguments, Trainer

# --- Re-instantiate the dataset ---
train_dataset = FlickrDataset(dataframe=df, processor=processor)

# --- Configure Training Arguments with Memory Optimizations ---
training_args = TrainingArguments(
    output_dir="blip-finetuned",
    learning_rate=5e-5,
    num_train_epochs=1,

    # --- MEMORY-SAVING CHANGES ---
    per_device_train_batch_size=4,  # Reduced batch size from 8 to 4
    gradient_accumulation_steps=2,  # Effectively a batch size of 4*2=8, but uses less memory
    fp16=True,                      # Use mixed-precision training to save memory
    # ---------------------------

    save_total_limit=2,
    push_to_hub=False,
    logging_steps=50,
    report_to="none",
)

# --- Initialize the Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

print("✅ Trainer has been re-initialized with memory-saving settings.")

✅ Trainer has been re-initialized with memory-saving settings.


In [ ]:
# --- Start the fine-tuning process ---
print("--- Starting the model fine-tuning... ---")
print("This will take some time. You will see the training loss decrease over time.")

trainer.train()

print("\n✅ Fine-tuning complete!")

--- Starting the model fine-tuning... ---
This will take some time. You will see the training loss decrease over time.


NameError: name 'trainer' is not defined

In [ ]:
# --- Define a path to save the final model ---
final_model_path = "./final_blip_model4"

# --- Save the model and the processor ---
print(f"\n--- Saving the final model and processor to {final_model_path} ---")
model.save_pretrained(final_model_path)
processor.save_pretrained(final_model_path)

print("\n✅ Model and processor have been saved successfully.")
print("You should now see a 'final_blip_model4' folder in your Colab file explorer on the left.")


--- Saving the final model and processor to ./final_blip_model3 ---

✅ Model and processor have been saved successfully.
You should now see a 'final_blip_model3' folder in your Colab file explorer on the left.


In [ ]:
!zip -r final_blip_model4.zip ./final_blip_model4

# from google.colab import files
# files.download('final_blip_model2.zip')

  adding: final_blip_model3/ (stored 0%)
  adding: final_blip_model3/generation_config.json (deflated 28%)
  adding: final_blip_model3/special_tokens_map.json (deflated 80%)
  adding: final_blip_model3/preprocessor_config.json (deflated 48%)
  adding: final_blip_model3/config.json (deflated 67%)
  adding: final_blip_model3/tokenizer_config.json (deflated 74%)
  adding: final_blip_model3/tokenizer.json (deflated 71%)
  adding: final_blip_model3/vocab.txt (deflated 53%)
  adding: final_blip_model3/model.safetensors (deflated 7%)
